In [1]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import clear_output
import jupyros as jr
import rospy
import os
from std_msgs.msg import String
from ass3.srv import Cordinates_srv    #used in case 1
from IPython.display import display
from geometry_msgs.msg import Twist, Vector3    #for cmd_vel topic
from jupyros import ros3d
import matplotlib.pyplot as plt
import tf
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
rospy.init_node('jupyter_user_interface')

task = []

In [2]:
#This cell has all the widgets
choice = widgets.Dropdown(
    options=['0','1', '2', '3'],
    value='0',
    description='Number:',
)

box_x = widgets.FloatText(
        value=-3.0,
        description='x:',
)

box_y = widgets.FloatText(
        value=5,
        description='Y:',
)

buttonUP = widgets.Button(
    icon="arrow-up"
)

buttonDOWN = widgets.Button(
    icon="arrow-down"
)

buttonLEFT = widgets.Button(
    icon="arrow-left"
)

buttonRIGHT = widgets.Button(
    icon="arrow-right"
)

buttonSTOP = widgets.Button(
    icon="stop"
)

buttonempty = widgets.Button(
    icon=""
)

arrow_box = widgets.HBox(
    [widgets.VBox([buttonempty,  buttonLEFT,  buttonempty]),
    widgets.VBox( [buttonUP,     buttonSTOP, buttonDOWN]),
    widgets.VBox( [buttonempty, buttonRIGHT, buttonempty])]
)
buttonenter = widgets.Button(
    description='Enter'
)
buttonenter2 = widgets.Button(
    description='Enter'
)

In [3]:
def sendtraget(b):
    print("Waiting for ROS!")
    rospy.wait_for_service('cordinates_srv')
    print("Values Sent!")
    cordinates_srv = rospy.ServiceProxy('cordinates_srv', Cordinates_srv)
    rt = cordinates_srv(box_x.value , box_y.value)
    if rt.return_ == 1:
        print("Target reached successfully!")
        task.append('Reached %')
    else:
        print("Target not reached")
        task.append('Not Reached %')
    
def case1():
    print("Please Enter x and y coordinates:")
    display(box_x)
    display(box_y)
    display(buttonenter)
    buttonenter.on_click(sendtraget)

In [4]:
def on_button_clicked_UP(b):
    moveingfn(1)
def on_button_clicked_LEFT(b):
    moveingfn(2)
def on_button_clicked_RIGHT(b):
    moveingfn(3)
def on_button_clicked_DOWN(b):
    moveingfn(4)
def on_button_clicked_STOP(b):
    moveingfn(5)

def moveingfn(direction):
    
    init = Vector3(0, 0, 0)
    vel = Twist( init, init)
    
    if direction == 1:
        #Front direction
        vel.linear.x = 0.5
        vel.angular.z = 0.0
    elif direction == 2:
        #Left direction
        vel.linear.x = 0.0
        vel.angular.z = 0.5
    elif direction == 3:
        #Right direction
        vel.linear.x = 0.0
        vel.angular.z = -0.5
    elif direction == 4:
        #Back direction
        vel.linear.x = -0.5
        vel.angular.z = -0.0
    elif direction ==5:
        #stop
        vel.linear.x = -0.0
        vel.angular.z = -0.0
        
    pub = rospy.Publisher('remap_cmd_vel',Twist, queue_size=10)
    pub.publish(vel)

In [5]:
def case2():
    display(arrow_box)
    buttonUP.on_click(on_button_clicked_UP)
    buttonDOWN.on_click(on_button_clicked_DOWN)
    buttonLEFT.on_click(on_button_clicked_LEFT)
    buttonRIGHT.on_click(on_button_clicked_RIGHT)
    buttonSTOP.on_click(on_button_clicked_STOP)

In [8]:
class robotposion:
    index = 0
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'g>')
        self.x_data, self.y_data = [] , []
    def plot_init(self):
        self.ax.set_xlim( -10, 10)
        self.ax.set_ylim( -10, 10)
        return self.ln
    def odom_callback(self, msg):
        self.index = self.index+1
        if self.index == 250:
            if len(self.y_data) < 500:
                self.y_data.append(msg.pose.pose.position.y)
            else:
                self.y_data.pop(0)
                self.y_data.append(msg.pose.pose.position.y)

            if len(self.x_data) < 500:
                self.x_data.append(msg.pose.pose.position.x)
            else:
                self.x_data.pop(0)
                self.x_data.append(msg.pose.pose.position.x)
            self.index = 0
    def update_plot(self, frame1):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
vispos = robotposion()
rospy.Subscriber('/odom', Odometry, vispos.odom_callback)
ani1 = FuncAnimation(vispos.fig, vispos.update_plot, init_func=vispos.plot_init, blit=True)
plt.title("Robot Motion")
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
class robotlaser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'bo')
        self.x_data, self.y_data = [] , []
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln
    def laser_callback(self, msg):
        print(vislas.x_data,vislas.y_data)
        increment = (msg.angle_max - msg.angle_min)/len(msg.intensities) 
        for i in range(len(msg.ranges)):
            x = msg.ranges[i]*np.cos(msg.angle_min + increment*i)
            y = msg.ranges[i]*np.sin(msg.angle_min + increment*i)
            self.y_data.append(y)
            self.x_data.append(x)
    def update_plot(self, frame2):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
vislas = robotlaser()
rospy.Subscriber('/scan', LaserScan, vislas.laser_callback)
ani2 = FuncAnimation(vislas.fig, vislas.update_plot, init_func=vislas.plot_init)
plt.title("Laser Vision")
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
fig, ax = plt.subplots()
_, _, ln, = ax.hist([])
def init():
    return ln,
def update(sth):
    ax.hist(task, bins=3, color='blue')
    return ln,
ani3 = FuncAnimation(fig, update, init_func=init, blit=True)
plt.show()
plt.title("Tragets Reached")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Tragets Reached')

In [9]:
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090")
tf_client = ros3d.TFClient(ros=rc, fixed_frame='link_chassis')
laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client)
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc, 
tf_client=tf_client)
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client, 
path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))
v.objects = [laser_view, map_view, path, urdf]
v

Viewer(objects=[LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_frame='link_c…

In [ ]:
#This cell is for the main interface
def startprogram(b):
    clear_output()
    print("Please enter your choice:")
    print("1) autonomously reach a x,y coordinate provided by the user")
    print("2) drive the robot using the keyboard")
    print("3) drive the robot using the keyboard with collisions avoidance")
    display(choice)
    display(buttonenter2)
    buttonenter2.on_click(startprogram)
    print("your choice is:")
    if (choice.value == '1'):
        #case one
        print("Case one")
        case1()
    elif (choice.value == '2'):
        #case two
        print("Case two")
        rospy.set_param("/obstacle_avoidance", 0)
        case2()
    elif (choice.value == '3'):
        #case three
        print("Case Three")
        rospy.set_param("/obstacle_avoidance", 1)
        case2()
    
print("Please enter your choice:")
print("1) autonomously reach a x,y coordinate provided by the user")
print("2) drive the robot using the keyboard")
print("3) drive the robot using the keyboard with collisions avoidance")
display(choice)
display(buttonenter2)
buttonenter2.on_click(startprogram)